## Libraries and SAM Initialization

In [ ]:
import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())
import sys
# !{sys.executable} -m pip install opencv-python matplotlib
# !pip install git+https://github.com/facebookresearch/segment-anything.git

In [261]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import shutil

In [262]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_b_01ec64.pth"
model_type = "vit_b"


device = "cpu"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

## Functions

In [263]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

In [264]:
def pltshow(masks,image,title='',annotations=False,size=5):
    plt.figure(figsize=(size,size))
    plt.imshow(image)
    if(annotations):
        show_anns(masks)
    plt.axis('off')
    if(title!=''):
        plt.title(title)
    plt.show()


In [265]:
def returnMasks(image):
    mask_generator_2 = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=20,
    pred_iou_thresh=0.85,
    stability_score_thresh=0.85,
    min_mask_region_area=300,
    stability_score_offset = 1.0,
    box_nms_thresh = 0.7,
    )
    masks = mask_generator_2.generate(image)
    return masks

In [266]:
def cutmask(greater,smaller):
    return cv2.bitwise_and(greater,cv2.bitwise_not(smaller))

In [267]:
def maskArea(mask):
    return cv2.countNonZero(mask)

## Removing small masks

##### We remove small masks since there is a high probability of them being miscategorized. We then add all these small masks to the cell mask.

In [268]:
def removeSmallMasks(masks,min_area):
    topop=[]
    for i,mask in enumerate(masks):
        if maskArea(mask)<min_area:
            topop.append(i)
    topop.sort(reverse=True)
    small_masks = []
    for pos in topop:
        small_masks.append(masks[pos])
        del masks[pos]
    return masks,small_masks

## Seperating overlapping area in different masks

In [269]:
def disjoinMasks(masks):
    newmasks=[]
    for mask in masks:
        newmasks.append((mask['segmentation'] * 255).astype(np.uint8))
    for i,mask1 in enumerate(newmasks):      
        
        for j,mask2 in enumerate(newmasks):    
                    
            areaM1=maskArea(mask1)
            areaM2=maskArea(mask2)
            
            if areaM1==areaM2:
                continue
            elif areaM1>areaM2:
                newmasks[i]=cutmask(mask1,mask2) 
            elif areaM1<areaM2:
               newmasks[j]=cutmask(mask2,mask1)
    newmasks,small_masks=removeSmallMasks(newmasks,600)
    return newmasks,small_masks

## Identifying and creating mask for bony-trabeculae

In [270]:
def identify_bony_trabeculae(image,masksDisjoint,output_dir):
    orange_lower = np.array([0, 50, 20], dtype=np.uint8)
    orange_upper = np.array([100, 255, 255], dtype=np.uint8)
    black_lower = np.array([0, 0, 0], dtype=np.uint8)
    black_upper = np.array([30, 30, 30], dtype=np.uint8)
    
    accepted_masks_indices = []
    combined_bony_mask = None
    combined_bony_mask_binary = None
    mask_number=0
    bone_dir = output_dir+'/bony-trabeculae'
    for mask_index, mask in enumerate(masksDisjoint):
        result_image = cv2.bitwise_and(image, image, mask=mask) # superimposing otiginal colors on the mask
        orange_mask = cv2.inRange(result_image, orange_lower, orange_upper)
        black_mask = cv2.inRange(result_image, black_lower, black_upper)
        no_pixels = np.sum(orange_mask > 0)
        nb_pixels = np.sum(black_mask > 0)
        height, width = image.shape[:2]
        total_area = height * width
        area_excluding_black = total_area - nb_pixels
        result_image = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)

        if ((no_pixels / area_excluding_black) * 100) >= 45:
            if combined_bony_mask is None:
                combined_bony_mask = result_image
                combined_bony_mask_binary = mask
                cv2.imwrite(bone_dir+"/mask-"+str(mask_number)+".png",mask) # saving individual masks
                mask_number+=1
            else:
                combined_bony_mask = cv2.bitwise_or(combined_bony_mask, result_image)
                combined_bony_mask_binary = cv2.bitwise_or(combined_bony_mask_binary, mask)
                cv2.imwrite(bone_dir+"/mask-"+str(mask_number)+".png",mask) # saving individual masks
                mask_number+=1

            accepted_masks_indices.append(mask_index)
    combined_bony_mask_copy = combined_bony_mask.copy()
    combined_bony_mask_copy = cv2.cvtColor(combined_bony_mask_copy, cv2.COLOR_BGR2RGB)
    cv2.imwrite(output_dir+"/bone-mask"+".tif",combined_bony_mask_copy) # saving combined masks
    # Removing masks from masksDisjoint that have been identified as fat.
    for index in sorted(accepted_masks_indices, reverse=True):
        del masksDisjoint[index]

    return masksDisjoint

## Identifying and creating mask for fat

In [271]:
def identify_fat(image,masksDisjoint,output_dir):
    black_lower = np.array([0, 0, 0], dtype=np.uint8)
    black_upper = np.array([30, 30, 30], dtype=np.uint8)
    accepted_masks_indices = []
    combined_fat_mask = None
    combined_fat_mask_binary = None
    mask_number=0
    fat_dir = output_dir+'/fat'
    for mask_index,mask in enumerate(masksDisjoint):
        result_image = cv2.bitwise_and(image, image, mask=mask) # superimposing otiginal colors on the mask
        black_mask = cv2.inRange(result_image, black_lower, black_upper)
       
        # Invert the black mask to exclude black pixels
        non_black_mask = cv2.bitwise_not(black_mask)
       
        # Compute mean and standard deviation of pixel intensities excluding black pixels
        mean, std_devs = cv2.meanStdDev(result_image, mask=non_black_mask)
       
        # Flatten the array of standard deviations
        std_devs = std_devs.flatten()
       
        # Compute the average standard deviation
        average_std_dev = np.mean(std_devs)
        if average_std_dev < 20:
            # Combine the selected masks into one mask using bitwise OR
            if combined_fat_mask is None:
                combined_fat_mask = result_image
                combined_fat_mask_binary = mask
                cv2.imwrite(fat_dir+"/mask-"+str(mask_number)+".png",mask) # saving individual masks
                mask_number += 1
            else:
                combined_fat_mask = cv2.bitwise_or(combined_fat_mask, result_image)
                combined_fat_mask_binary = cv2.bitwise_or(combined_fat_mask_binary, mask)
                cv2.imwrite(fat_dir+"/mask-"+str(mask_number)+".png",mask) # saving individual masks
                mask_number += 1
            accepted_masks_indices.append(mask_index)
    

    cv2.imwrite(output_dir+"/fat-mask"+".tif",combined_fat_mask) # saving combined masks
    # Removing masks from masksDisjoint that have been identified as fat.
    for index in sorted(accepted_masks_indices, reverse=True):
        del masksDisjoint[index]

    return masksDisjoint

## Creating mask for cellular area(including reticulin)

In [272]:
def identify_cell(image,masksDisjoint,output_dir):
    combined_cell_mask = None
    cell_dir = output_dir + "/cell"
    mask_number = 0
    for mask in masksDisjoint:
        result_image = cv2.bitwise_and(image, image, mask=mask) # superimposing otiginal colors on the mask
        result_image = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)
        if combined_cell_mask is None:
            combined_cell_mask = result_image
            cv2.imwrite(cell_dir+"/mask-"+str(mask_number)+".png",mask) # saving individual masks
            mask_number += 1
        else:
            combined_cell_mask = cv2.bitwise_or(combined_cell_mask, result_image) # saving individual masks
            cv2.imwrite(cell_dir+"/mask-"+str(mask_number)+".png",mask)
            mask_number += 1
    
    cv2.imwrite(output_dir+"/cell-mask"+".tif",combined_cell_mask) # saving combined masks
    return combined_cell_mask

In [273]:
def calculate_elongation(rect):
            """Calculate elongation from rotated rectangle."""
            width, height = rect[1]
            if width == 0 or height == 0:
                return 0  # Return 0 elongation if width or height is zero
            else:
                return 1 - (min(height, width) / max(height, width))

## Conversion from images to classes

##### Once we have images that require automatic annotation, SAM segments these images into regions using binary masks known as 'segmentation objects.' Each mask represents a region identified by SAM. However, these masks do not inherently classify regions into specific categories. To achieve meaningful segmentation, we need to assign class labels to these regions based on their characteristics. Let's examine how SAM converts images into segmentation objects and how we can classify the masks based on their unique characteristics.

In [274]:
import os
input_directory_path="IMAGES" #input direcory contating images
output_directory_path = "masks" #output directory that will contain classified segmentations
for n, imagefile in enumerate(os.listdir(input_directory_path)):
    image_path = os.path.join(input_directory_path, imagefile)
    image = cv2.imread(image_path)

    image_name=imagefile.split(".")[0]
    output_dir=output_directory_path+'/'+image_name
    

    # Our annotations will first be saved in a folder annotations for all images. The structure of this folder will be:
    #Image Name
     #-->bony-trabeculae
     #   /-->mask-0.png
     #   /-->mask-1.png
     #
     #fat
     #   /-->mask-0.png
     #   /-->mask-1.
     #
     #cell
     #   /-->mask-0.png
     #   /-->mask-1.png
     #
     #reticulin
     #   /-->mask.png
     #
     #-->bone-mask.tif
     #-->fat-mask.tif
     #-->cell-mask.tif
     #-->original-image.tif


   
    os.mkdir(output_dir)
    os.mkdir(output_dir+'/bony-trabeculae')
    os.mkdir(output_dir+'/reticulin')
    os.mkdir(output_dir+'/fat')
    os.mkdir(output_dir+'/cell')
    cv2.imwrite(output_dir+"/original-image"+".tif",image) # saving the original image

    masks = returnMasks(image) # generating masks using SAM, the output is a list of binary image with white region as the segmented area/foreground and black being the rest of the image/background

    

    masksDisjoint,small_masks = disjoinMasks(masks)
    masksDisjoint = identify_bony_trabeculae(image, masksDisjoint,output_dir)
    
    masksDisjoint = identify_fat(image,masksDisjoint,output_dir)
    masksDisjoint = np.concatenate((masksDisjoint,small_masks),axis=0)
    combined_cell_mask = identify_cell(image,masksDisjoint,output_dir)
    
  

##### Now, by taking a look at the combined masks of each class for every image, we ensure that the classification is correct. If there is a misclassification, we move that individual mask to the directory of the correct class.

In [275]:
def combineBone(dir):
    combined_mask = None
    for n,file_name in enumerate(os.listdir(dir)):
        mask_path = os.path.join(dir, file_name)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        if combined_mask is None:
            
            combined_mask = mask
        else:
            combined_mask = cv2.bitwise_or(combined_mask,mask)

    shutil.rmtree(dir)
    os.mkdir(dir)
    if combined_mask is not None:
        cv2.imwrite(dir+"/bone-mask"+".png",combined_mask)

In [276]:
def combineFat(dir):
    combined_mask = None
    for n,file_name in enumerate(os.listdir(dir)):
        mask_path = os.path.join(dir, file_name)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        if combined_mask is None:
            combined_mask = mask
        else:
            combined_mask = cv2.bitwise_or(combined_mask,mask)

    shutil.rmtree(dir)
    os.mkdir(dir)
    if combined_mask is not None:
        cv2.imwrite(dir+"/fat-mask"+".png",combined_mask)

In [277]:
def combineCell(dir,b_dir):
    combined_mask = None
    for n,file_name in enumerate(os.listdir(dir)):
        mask_path = os.path.join(dir, file_name)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        if combined_mask is None:
            combined_mask = mask
        else:
            combined_mask = cv2.bitwise_or(combined_mask,mask)

    plt.imshow(combined_mask,cmap="gray")
    plt.show()
    shutil.rmtree(dir)
    os.mkdir(dir)
    if combined_mask is not None:
        cv2.imwrite(dir+"/cell-mask"+".png",combined_mask)


    # in some cases it may be that sam was unable to generate mask for cell, in that case we combine bone and fat masks and invert them to get the cell mask
    # this is not required for every image, so if not required comment out
    # Load the original image to get its dimensions
    og = cv2.imread(b_dir+"/original-image.tif", cv2.IMREAD_GRAYSCALE)
      
    # Create bone and fat masks
    bone = np.zeros_like(og)
    fat = np.zeros_like(og)

    # Check if bone mask exists, if not, it will remain all black
    bone_path = b_dir + "/bony-trabeculae/bone-mask.png"

    if os.path.exists(bone_path):
        bone = cv2.imread(bone_path, cv2.IMREAD_GRAYSCALE)

    # Check if fat mask exists, if not, it will remain all black
    fat_path = b_dir + "/fat/fat-mask.png"
    
    if os.path.exists(fat_path):
        fat = cv2.imread(fat_path, cv2.IMREAD_GRAYSCALE)


    total_cell = cv2.bitwise_or(bone,fat)
    total_cell = cv2.bitwise_not(total_cell)
    cv2.imwrite(dir+"/cell-mask-2"+".png",total_cell)

    # we now manually verify the two cell masks and delete one of them to avoid repition. If one of them is incorrect then we delete the incorrect mask
    # we rename this mask as cell-mask.png



## Combining the individual masks

In [ ]:
Masks = "masks" # input directory
for n,dir in enumerate(os.listdir(Masks)):
    subdir_path = os.path.join(Masks, dir)

    for image_folder in os.listdir(subdir_path):
        f_dir = os.path.join(subdir_path, image_folder)
        if image_folder == "bony-trabeculae":
            if len(os.listdir(f_dir)) != 0:
                combineBone(f_dir)
            
        elif image_folder == "fat":
            if len(os.listdir(f_dir)) != 0:
                combineFat(f_dir)
    f_dir = os.path.join(subdir_path, "cell")

    if len(os.listdir(f_dir)) != 0:
        combineCell(f_dir,subdir_path)

## Isolating Reticulin Fibers

In [279]:
def identify_reticulin(dir):
    mask = os.listdir(dir+"/cell")
    print(mask)
    cell_mask = cv2.imread(dir+"/cell/"+mask[0]) 
    print(dir+"/cell/"+mask[0])
    print(os.path.exists(dir+"/"+mask[0]))
    plt.imshow(cell_mask,cmap="gray")
    plt.show()
    # o_ret = cv2.imread(dir+"/reticulin/mask.png") 
    o_img = cv2.imread(dir+"/original-image.tif")
    bone = np.zeros((cell_mask.shape[0], cell_mask.shape[1]), dtype=cell_mask.dtype)
    fat = np.zeros((cell_mask.shape[0], cell_mask.shape[1]), dtype=cell_mask.dtype)

    o_img = cv2.cvtColor(o_img, cv2.COLOR_BGR2RGB)
    # Check if bone mask exists, if not, it will remain all black
    bone_path = dir + "/bony-trabeculae/bone-mask.png"
    if os.path.exists(bone_path):
        bone = cv2.imread(bone_path, cv2.IMREAD_GRAYSCALE)


    # Check if fat mask exists, if not, it will remain all black
    fat_path = dir + "/fat/fat-mask.png"
    if os.path.exists(fat_path):
        fat = cv2.imread(fat_path, cv2.IMREAD_GRAYSCALE)

    b_mask = cv2.bitwise_not(bone)
    fat_mask = cv2.bitwise_not(fat)
    b_mask = cv2.bitwise_or(bone,fat)

    b_mask = cv2.bitwise_not(b_mask)
    test = o_img
    # as the cell mask will be black in places where bone and fat are present, so we need to convert that area into white before thresholding
    test[b_mask == 0] = 255,255,255

    grayscale_image = cv2.cvtColor(test, cv2.COLOR_BGR2GRAY)



    # Apply manual binary thresholding
    # Adjust the value of the threshold to ensure maximum reticulin identification 
    _, binary_image = cv2.threshold(grayscale_image, 155, 255, cv2.THRESH_BINARY)
    binary_image= cv2.bitwise_not(binary_image)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create a copy of the thresholded image for drawing contours
    contours_image =cv2.bitwise_not(binary_image)

    contours_image = cv2.cvtColor(contours_image,cv2.COLOR_GRAY2BGR)

    # Convert BGR to RGB
    contours_image_rgb = cv2.cvtColor(contours_image, cv2.COLOR_BGR2RGB)    

    # Display using matplotlib
    # plt.figure(figsize=(10, 8))  # Adjust figure size as needed
    
    accepted_contours = []
    rejected_contours = []

    # Loop over the contours
    for contour in contours:
        # Fit a rotated rectangle around the contour
        rect = cv2.minAreaRect(contour)
        
        # Calculate elongation
        elongation = calculate_elongation(rect)       
           



        if elongation >= 0.25:
            accepted_contours.append(contour)
        else:
            rejected_contours.append(contour)
    
        

    # Create masks for accepted and rejected contours
    accepted_contours_mask = np.zeros_like(binary_image)
    rejected_contours_mask = np.zeros_like(binary_image)

    # Draw contours onto masks
    for contour in accepted_contours:
        cv2.drawContours(accepted_contours_mask, [contour], 0, (255), cv2.FILLED)

    for contour in rejected_contours:
        cv2.drawContours(rejected_contours_mask, [contour], 0, (255), cv2.FILLED)

    # Invert the rejected contours mask
    rejected_contours_mask = cv2.bitwise_not(rejected_contours_mask)
    
    # Perform bitwise AND operation with the binary image and rejected contours mask
    final_image = cv2.bitwise_and(binary_image, binary_image, mask=rejected_contours_mask)
    tem = cv2.bitwise_not(final_image)

    # # Plot both images side by side
    fig, axes = plt.subplots(1, 2, figsize=(18, 7))
    o_img = cv2.imread(dir+"/original-image.tif")
    o_img = cv2.cvtColor(o_img, cv2.COLOR_BGR2RGB)
    # Plot the original image
    axes[0].imshow(o_img)
    axes[0].set_title('Original Image')

    # Plot the thresholded image
    axes[1].imshow(tem,cmap="gray")
    axes[1].set_title('Threshold Image')
    # Hide the axes
    for ax in axes:
        ax.axis('off')

    # Display the plot
    plt.show()


    shutil.rmtree(dir+"/reticulin")
    os.mkdir(dir+"/reticulin")
    cv2.imwrite(dir+"/reticulin/mask.png",final_image)

In [ ]:
Masks = "masks" #Directory containing all masks
for n, dir in enumerate(os.listdir(Masks)):
    subdir_path = os.path.join(Masks, dir)
    identify_reticulin(subdir_path)


## Converting masks to annotations

##### We use an annotation platform called CVAT to edit our annotations. This platform requires annotations in a standard format such as PascalVOC, COCO, or Cityscapes. PascalVOC is an annotation format which is popular and easy to manipulate, as it saves annotations in the form of images, rather than boundary pixel coordinates in XML files or JSON files used by the COCO format. The following is the directory structure of PascalVOC:

- Annotations
    - Image_Name_1.xml
    - Image_Name_2.xml
- ImageSets
    - Main
        - default.txt
    - Segmentation
        - default.txt
- JPEGImages (optional)
    - Image_Name_1.jpg
    - Image_Name_2.jpg        
- SegmentationClass
    - Image_Name_1.jpg
    - Image_Name_2.jpg
- SegmentationObject
    - Image_Name_1.png
    - Image_Name_2.png
- lablmap.txt


Contents of each file/folder
- Annotations: Contains XML files that provide basic information for each image, such as filename, width, and height.
- ImageSets: Contains two TXT files under Main and Segmentation. These files list the names of each image.
- JPEGImages (optional): This folder contains all the original images.
- SegmentationClass: A folder containing PNG files of SegmentationClass images. Each pixel in these images represents a class with a specific color. For example, cells, background (bg), bone, and fat each have distinct colors. There are only four colors in these images.
- SegmentationObject: A folder containing PNG files of SegmentationObject images. Each distinct object recognized in the images is assigned a different color. For instance, fat1, fat2, bone1, and bone2 are different objects within their respective classes, each colored uniquely. These images can use up to 256 different colors, the maximum limit of PascalVOC.
- lablmap.txt: This file specifies the color of each class in SegmentationClass. For example, fat:0,128,0 corresponds to the color representation of cells.


In [281]:
from lxml import etree

In [282]:
#Making the Xml file like the following example:
#     <annotation>
#        <folder/>
#            <filename>PGI_image_0.jpg</filename>
#        <size>
#            <width>1200</width>
#            <height>1920</height>
#            <depth/>
#        </size>
#        <segmented>1</segmented>
#     </annotation>5


def makeXML(img, width, height):
    annotation = etree.Element('annotation')

    fo = etree.Element('folder')
    fo.text =  ''

    annotation.append(fo)

    f = etree.Element('filename')
    f.text = str(img+'.jpg')

    annotation.append(f)

    size = etree.Element('size')
    w = etree.Element('width')
    w.text = str(width)
    h = etree.Element('height')
    h.text = str(height)
    d = etree.Element('depth')
    d.text = str('')


    size.append(w)
    size.append(h)
    size.append(d)
    annotation.append(size)

    segmented=etree.Element('segmented')
    segmented.text='1'
    annotation.append(segmented)


    return annotation

In [283]:
cleanOutput='./masks' # input the folder where we saved our annotations  
OutputDir='PASCALoutput/' # Folder that will contain the PascalVOC annotation
imagesFolder='IMAGES/' # Folder that will contains only images

# Create the directories
os.makedirs(imagesFolder, exist_ok=True)
os.makedirs(os.path.join(OutputDir, 'SegmentationObject'), exist_ok=True)
os.makedirs(os.path.join(OutputDir, 'SegmentationClass'), exist_ok=True)
os.makedirs(os.path.join(OutputDir, 'Annotations'), exist_ok=True)
os.makedirs(os.path.join(OutputDir, 'ImageSets', 'Main'), exist_ok=True)
os.makedirs(os.path.join(OutputDir, 'ImageSets', 'Segmentation'), exist_ok=True)

In [284]:
class_color={
    'bony-trabeculae':[128,128,0],
    'cell':[128,0,0],
    'reticulin':[0,0,128],
    'fat':[0,128,0]
}

In [285]:
default="" #default.txt file in 'ImageSets/Main' and 'ImageSets/Segmentation'

#EDIT THIS TO BE IN COORDINATION WITH class_color
labelmap="# label:color_rgb:parts:actions\nbackground:0,0,0::\ncell:128,0,0::\nfat:0,128,0::\nreticulin:0,0,128::\nbony-trabeculae:128,128,0::" #labelmap.txt file


In [286]:
def binarize_for_pascal(rgb_image):
    gray_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(gray_image, 15, 255, cv2.THRESH_BINARY)
    return binary_mask


In [287]:
def color_map_get256(N=256):
    def bitget(byteval, idx):
        return ((byteval & (1 << idx)) != 0)

    cmap = np.zeros((N, 3), dtype=np.dtype)
    for i in range(N):
        r = g = b = 0
        c = i
        for j in range(8):
            r = r | (bitget(c, 0) << 7-j)
            g = g | (bitget(c, 1) << 7-j)
            b = b | (bitget(c, 2) << 7-j)
            c = c >> 3

        cmap[i] = np.array([r, g, b])
    return cmap

color_map256=color_map_get256()

In [288]:
def addObject2Image(image,objects,color): # add an object to image with random color
    masked_image=image.copy()
    masked_image[objects==255]=color[::-1]
    return masked_image

In [289]:
def addClass2Image(image,mask,color_rgb):
    color_bgr=color_rgb[::-1]
    masked_image=image.copy()
    masked_image[mask==255]=color_bgr
    return masked_image

In [ ]:
default="" #making sure default file is empty before appending
outofLimitMaskClasses=[]
for n,dir in enumerate(os.listdir(cleanOutput)): #dir=MQ_image_0 or PGI_image_0
        # print(dir)
        default+=dir+"\n" #adding image name to default.txt

        og_img=cv2.imread(cleanOutput+'/'+dir+'/original-image.tif')
        cv2.imwrite(imagesFolder+'/'+dir+'.tif',og_img) #adding image to image directory for data to be uploaded later

        #making annotation xml file for each image
        og_image_shape=og_img.shape
        annotation=makeXML(dir,og_image_shape[0],og_image_shape[1])
        save_path = os.path.join(OutputDir+'/Annotations/'+dir+ ".xml")
        with open(save_path, 'wb') as file:
            aux = etree.tostring(annotation, pretty_print=True)
            aux.decode("utf-8")
            file.write(aux)

        image_objects=np.zeros(og_img.shape) #initiallly we use empty mask before overlaying individual masks on it
        image_classes=np.zeros(og_img.shape) #initiallly we use empty mask before overlaying combined class mask on it
        empty_mask=np.zeros_like(cv2.cvtColor(og_img,cv2.COLOR_BGR2GRAY)) #to save combined mask for each class

        combined_class={
            'bony-trabeculae': empty_mask,
            'reticulin': empty_mask,
            'fat': empty_mask,
            'cell': empty_mask
        }

        color_number_for_mask=2 #start colouring masks from color no. 2 in color_map256

        for item in os.listdir(cleanOutput+'/'+dir): # bone,cell,og.jpg
            item_path = os.path.join(cleanOutput+'/'+dir, item) # '/Individual-Masks/MQ_image_0/bone'
            if os.path.isdir(item_path): 
                mask = os.listdir(item_path)
                if mask:

                    mask=cv2.imread(item_path+'/'+mask[0])
                    binary_mask=binarize_for_pascal(mask)

                    combined_class[item]=cv2.bitwise_or(combined_class[item],binary_mask)  #combine each class's mask

                    image_objects=addObject2Image(image_objects,binary_mask,color_map256[color_number_for_mask])
                    
                    color_number_for_mask+=1
   



        for mask_class in ['cell','reticulin','fat','bony-trabeculae']:
            image_classes=addClass2Image(image_classes,combined_class[mask_class],class_color[mask_class])

        cv2.imwrite(OutputDir+'/SegmentationObject/'+dir+'.png',image_objects)
        cv2.imwrite(OutputDir+'/SegmentationClass/'+dir+'.png',image_classes)

        print(n,end="-")

In [291]:
text_file = open(OutputDir+'/ImageSets/Main/default.txt', "w")
text_file.write(default)
text_file.close()

text_file = open(OutputDir+'/ImageSets/Segmentation/default.txt', "w")
text_file.write(default)
text_file.close()

text_file = open(OutputDir+'/labelmap.txt', "w")
text_file.write(labelmap)
text_file.close()